In [ ]:
from google.colab import files

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
import numpy as py

import os
import shutil

In [ ]:
!pip install -q kaggle

In [ ]:
files.upload()

Saving kaggle.json to kaggle (4).json


{'kaggle.json': b'{"username":"bayuabdurrosyid","key":"377e83e17d65f0c8a79606180f4883c4"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d madisona/translated-animals10

401 - Unauthorized


In [ ]:
!mkdir animals
!unzip -qq translated-animals10.zip -d animals

In [ ]:
dir = '/content/animals/animals10/raw-img'

In [ ]:
ignore = ['squirrel', 'spider', 'butterfly', 'chicken', 'elephant', 'horse']

for x in ignore:
  path = os.path.join(dir, x)
  shutil.rmtree(path)

print(os.listdir(dir))

['sheep', 'cow', 'cat', 'dog']


In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode = 'nearest',
    horizontal_flip = True,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.15,
    validation_split = 0.2
)

In [ ]:
w = 150
h = 150

train_dataset = datagen.flow_from_directory(
    dir,
    class_mode='categorical',
    subset = 'training',
    target_size = (w, h),
    batch_size = 256
)

validation_dataset = datagen.flow_from_directory(
    dir,
    class_mode='categorical',
    subset = 'validation',
    target_size = (w, h),
    batch_size = 256
)

Found 8175 images belonging to 4 classes.
Found 2042 images belonging to 4 classes.


In [ ]:
model = Sequential(
    [
     layers.Conv2D(64, (3,3), activation='relu', input_shape=(w, h, 3)),
     layers.MaxPooling2D(2, 2),
     layers.Conv2D(64, (3, 3), activation='relu'),
     layers.MaxPooling2D(2, 2),
     layers.Conv2D(128, (3, 3), activation='relu'),
     layers.MaxPooling2D(2, 2),
     layers.Conv2D(256, (3, 3), activation='relu'),
     layers.MaxPooling2D(2, 2),
     layers.Dropout(0.2),
     layers.Flatten(),
     layers.Dense(512, activation = 'relu'),
     layers.Dense(256, activation = 'relu'),
     layers.Dense(4, activation = 'softmax')
    ]
)

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [ ]:
class callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epocj, logs = None):
    if (logs.get('accuracy') > 0.92 and logs.get('val_accuracy') > 0.92):
      print('Akurasi telah mencapai 92%!')
      self.model.stop_training = True

In [ ]:
history = model.fit(
    train_dataset,
    validation_data = validation_dataset,
    validation_steps = 5,
    epochs = 40,
    verbose = 2,
    callbacks = [callback()]    
)

Epoch 1/40
32/32 - 68s - loss: 1.3140 - accuracy: 0.4572 - val_loss: 1.2318 - val_accuracy: 0.4734 - 68s/epoch - 2s/step
Epoch 2/40
32/32 - 66s - loss: 1.1686 - accuracy: 0.5009 - val_loss: 1.0565 - val_accuracy: 0.5227 - 66s/epoch - 2s/step
Epoch 3/40
32/32 - 65s - loss: 1.0435 - accuracy: 0.5595 - val_loss: 0.9874 - val_accuracy: 0.5695 - 65s/epoch - 2s/step
Epoch 4/40
32/32 - 65s - loss: 1.0119 - accuracy: 0.5681 - val_loss: 0.9575 - val_accuracy: 0.5672 - 65s/epoch - 2s/step
Epoch 5/40
32/32 - 65s - loss: 0.9477 - accuracy: 0.5994 - val_loss: 0.9260 - val_accuracy: 0.6086 - 65s/epoch - 2s/step
Epoch 6/40
32/32 - 65s - loss: 0.9347 - accuracy: 0.6159 - val_loss: 0.8851 - val_accuracy: 0.6391 - 65s/epoch - 2s/step
Epoch 7/40
32/32 - 65s - loss: 0.8906 - accuracy: 0.6402 - val_loss: 0.8771 - val_accuracy: 0.6313 - 65s/epoch - 2s/step
Epoch 8/40
32/32 - 65s - loss: 0.8367 - accuracy: 0.6536 - val_loss: 0.8735 - val_accuracy: 0.6414 - 65s/epoch - 2s/step
Epoch 9/40
32/32 - 64s - loss: 0

In [ ]:
history = model.fit(
    train_dataset,
    validation_data = validation_dataset,
    validation_steps = 5,
    epochs = 40,
    verbose = 2,
    callbacks = [callback()]    
)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# with tf.io.gfile.GFile('model.tflite', 'wb') as f:
#   f.write(tflite_model)

In [ ]:
# !ls -la | grep 'model' 